In [1]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt

def calculate_qubit_1bus_1read_equiv_cap(C_M, index):
    """
    Calculate effective shunt and coupling capacitance.
    Arg:
        C_M : Maxewll capacitance matrix
        index : index of each conductor in C_M, starting from 0
    Return:
        Tuple of effective bus, readout and shunt capacitance
    """
    index_bus1 = index['bus1']
    index_gnd = index['gnd']
    index_island1 = index['island1']
    index_island2 = index['island2']
    index_read = index['read']
    
    NUM_CONDUCTOR = C_M.shape[0]
    
    C_M_1 = np.zeros((NUM_CONDUCTOR, NUM_CONDUCTOR))
    C_M_1[:, index_island1] = C_M[:, index_island1]
    C_M_1[:, index_island2] = C_M[:, index_island2]
    C_M_1[index_bus1, index_bus1] = -1
    C_M_1[index_gnd, index_gnd] = -1
    C_M_1[index_read, index_read] = -1

    b1 = np.linalg.solve(C_M_1, -C_M[:, index_bus1])
    beta_bus1 = b1[index_island1] - b1[index_island2]

    r = np.linalg.solve(C_M_1, -C_M[:, index_read])
    beta_read = r[index_island1] - r[index_island2]
    
    # Find C_sigma
    # V_island1 - V_island2 = 1, Eq.1
    # Q_island1 + Q_island2 = 0, Eq.2
    # From Eq1 and Eq2, find V_island1 and V_island2 

    A = np.array([[1, -1],
                 [C_M[index_island1, index_island1]+C_M[index_island2, index_island1]
                  , C_M[index_island1, index_island2]+C_M[index_island2, index_island2]]])

    b = np.array([[1]
                  ,[0]])
    V = np.linalg.solve(A, b)  # A*V = b, solve for V[0]=V_island1, V[1]=V_island2

    V0 = np.zeros(NUM_CONDUCTOR)
    V0[index_island1] = V[0]
    V0[index_island2] = V[1]
    V0[index_bus1] = 0
    V0[index_read] = 0
    V0[index_gnd] = 0

    Q = C_M.dot(V0)
    C_sigma = np.dot(Q, V0)
    
    # find C_bus1, C_read and C_shunt
    A = np.array([[beta_bus1-1, beta_bus1, beta_bus1]
                ,[beta_read,  beta_read-1, beta_read]  
                ,[1, 1, 1]])
    b = np.array([[0]
                  ,[0]
                 ,[C_sigma]])
    c = np.linalg.solve(A, b)
      
    return c

def update_mutual_capacitance(Maxwell_cap_matrix,conductor1_index, conductor2_index, new_cap):
    """
    Args:
        Maxwell_cap_matrix: Maxwell capacitance matrix from Q3D
        conductor1_index : index of conductor 1
        conductor2_index : index of conductor 2
        new_cap : new capacitance for replacement
    """
    
    if conductor1_index == conductor2_index:
        raise('Error. First and second conductor index must be different!')
    
    M = np.copy(Maxwell_cap_matrix)
    
    M[conductor1_index, conductor2_index] = -new_cap
    M[conductor2_index, conductor1_index] = -new_cap
    
    for i in range(M.shape[0]):
        M[i,i] = -(np.sum(M[i,:]) - M[i,i])
        
    return M

def print_result(c):
    c = np.ndarray.flatten(c)
    print(f'C_bus1 = {c[0]:.3f} fF\n' 
        + f'C_read = {c[1]:.3f} fF\n'
        + f'C_shunt = {c[2]:.3f} fF')

# 1-bus, 1-readout

In [2]:
# C_M : Maxell capacitance matrix from Q3D
C_M = np.array([[107.94, -96.56, -0.72528, -9.7239,-0.041219]
               ,[-96.56, 586.48,  -59.967, -55.133, -95.917]
               ,[-0.72528, -59.967, 92.027,  -23.543,   -7.0491]
               ,[-9.7239, -55.133, -23.543, 98.962,-0.61872]
               ,[-0.041219, -95.917,   -7.0491, -0.61872, 103.03]
              ])

# index dictionary of conductors in input Maxwell capacitance matrix
conductor_index = { 'gnd' : 1
                   , 'island1' : 2
                   , 'island2' : 3
                   , 'bus1' : 0
                   , 'read' : 4
                  }

In [4]:
cap_result = calculate_qubit_1bus_1read_equiv_cap(C_M, conductor_index)
print_result(cap_result)

C_bus1 = -4.248 fF
C_read = 3.400 fF
C_shunt = 60.283 fF


# update mutual capacitance. No need to change C_M manually

In [5]:
C_M_1 = update_mutual_capacitance(C_M, 0, 3, 22)

In [6]:
cap_result1 = calculate_qubit_1bus_1read_equiv_cap(C_M_1, conductor_index)
print_result(cap_result1)

C_bus1 = -9.856 fF
C_read = 3.479 fF
C_shunt = 66.121 fF
